In [2]:
import itertools

In [14]:
pv_template = (1,0,1,0,0,0,1,0,0,0,0,1,0,0)

In [8]:
def template_to_product(template):
  return map(lambda b: [0,1] if b == 0 else [0,1,2], template)

In [15]:
def pv_space_size(template):
  p = 1
  for b in template:
    if b == 0:
      p *= 2
    else:
      p *= 3
  return p

In [17]:
pv_space_size(pv_template)

82944

In [9]:
for pv in itertools.product(*template_to_product(pv_template)):
  print(pv)

(0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 1)
(0, 0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 0, 0, 1, 1)
(0, 0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 0, 0, 1, 0, 1)
(0, 0, 0, 0, 0, 1, 1, 0)
(0, 0, 0, 0, 0, 1, 1, 1)
(0, 0, 0, 0, 0, 2, 0, 0)
(0, 0, 0, 0, 0, 2, 0, 1)
(0, 0, 0, 0, 0, 2, 1, 0)
(0, 0, 0, 0, 0, 2, 1, 1)
(0, 0, 0, 0, 1, 0, 0, 0)
(0, 0, 0, 0, 1, 0, 0, 1)
(0, 0, 0, 0, 1, 0, 1, 0)
(0, 0, 0, 0, 1, 0, 1, 1)
(0, 0, 0, 0, 1, 1, 0, 0)
(0, 0, 0, 0, 1, 1, 0, 1)
(0, 0, 0, 0, 1, 1, 1, 0)
(0, 0, 0, 0, 1, 1, 1, 1)
(0, 0, 0, 0, 1, 2, 0, 0)
(0, 0, 0, 0, 1, 2, 0, 1)
(0, 0, 0, 0, 1, 2, 1, 0)
(0, 0, 0, 0, 1, 2, 1, 1)
(0, 0, 0, 1, 0, 0, 0, 0)
(0, 0, 0, 1, 0, 0, 0, 1)
(0, 0, 0, 1, 0, 0, 1, 0)
(0, 0, 0, 1, 0, 0, 1, 1)
(0, 0, 0, 1, 0, 1, 0, 0)
(0, 0, 0, 1, 0, 1, 0, 1)
(0, 0, 0, 1, 0, 1, 1, 0)
(0, 0, 0, 1, 0, 1, 1, 1)
(0, 0, 0, 1, 0, 2, 0, 0)
(0, 0, 0, 1, 0, 2, 0, 1)
(0, 0, 0, 1, 0, 2, 1, 0)
(0, 0, 0, 1, 0, 2, 1, 1)
(0, 0, 0, 1, 1, 0, 0, 0)
(0, 0, 0, 1, 1, 0, 0, 1)
(0, 0, 0, 1, 1, 0, 1, 0)
(0, 0, 0, 1, 1, 0, 1, 1)


In [10]:
def extended_gcd(a, b):
    """Extended Euclidean Algorithm.
    Returns (gcd, x, y) such that a*x + b*y = gcd"""
    if b == 0:
        return (a, 1, 0)
    else:
        g, x1, y1 = extended_gcd(b, a % b)
        return (g, y1, x1 - (a // b) * y1)

def crt(a, b, A, B):
    """Chinese Remainder Theorem for coprime A and B.
    Returns x such that x ≡ a mod A and x ≡ b mod B."""
    g, m1, m2 = extended_gcd(A, B)
    if g != 1:
        raise ValueError("A and B must be coprime")
    
    # Compute the solution modulo A * B
    x = (a * m2 * B + b * m1 * A) % (A * B)
    return x

# Example usage:
a, b, A, B = 1, 2, 2, 3
x = crt(a, b, A, B)
print(f"x ≡ {x} mod {A*B}")

x ≡ 5 mod 6


In [ ]:
def solve(right, bottom):
  """
  for a in itertools.product([0,1],[0,1,2]):
    print(a, solve(a[1],a[0]))
  """
  tile = crt(bottom, right, 2, 3)
  return tile//2, tile//3

def line(right_most, bottom):
  top = []

  curr_right = right_most
  for b in bottom:
    curr_right, curr_top = solve(curr_right, b)
    top.append(curr_top)

  return curr_right, top

In [18]:
print(line(1,[]))

(1, [])


In [ ]:
def rotate(l, n):
    return l[-n:] + l[:-n]

def next_cyclic_valuation_b6(pv,pv_template):
  bottom = []
  new_pv = []
  for i, b in enumerate(pv_template[::-1]):
    if b == 0:
      bottom.append(pv[::-1][i])
    if b == 1:
      left_most, top = line(pv[::-1][i], bottom[::-1])
      #print(pv[::-1][i], bottom[::-1], left_most, top)
      new_pv.append(left_most)
      new_pv += top[::-1]
      #print(new_pv)
      bottom = []
  #print(new_pv[::-1])
  return tuple(rotate(new_pv[::-1],1))

In [49]:
pv = pv_template
seen = set({})

while pv not in seen:
  seen.add(pv)
  pv = next_cyclic_valuation_b6(pv,pv_template)

len(seen)

90

In [50]:
next_of_pv = {}
for pv in itertools.product(*template_to_product(pv_template)):
  next_of_pv[pv] = next_cyclic_valuation_b6(pv,pv_template)

In [51]:
len(next_of_pv)

82944

In [61]:
def get_connex_components(next_graph):
  component_of = {}
  component_id = 0
  components = {}

  for e in list(next_graph):
    if e in component_of:
      continue
    if e not in component_of:
      component_of[e] = component_id
      if component_id not in components:
        components[component_id] = set({})
      components[component_id].add(e)
      e_next = next_graph[e]
      while e_next not in component_of:
        components[component_id].add(e_next)
        component_of[e_next] = component_id
        e_next = next_graph[e_next]
      component_id += 1

  return components, component_of

In [62]:
components, component_of = get_connex_components(next_of_pv)

In [56]:
len(components)

79573

In [65]:
component_of[pv_template]

31288

In [67]:
components[31288]

{(1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0),
 (1, 1, 2, 1, 0, 1, 2, 1, 0, 1, 0, 1, 1, 0)}